# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM DisasterDataset',engine)
X = df.message
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

In [4]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans(PUNCT_TO_REMOVE,' '*len(string.punctuation)))

In [5]:
def tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = lemmatize_words(text)
    return nltk.word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [7]:
pipeline = Pipeline(
    [
        ('vectorizer',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('classifer', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [8]:
print(pipeline)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,Y)

In [11]:
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
from sklearn.metrics import classification_report

In [13]:
y_pred = pipeline.predict(x_test)

In [14]:
def classification_report_for_each_column(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('-------:',col,':-------')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

In [15]:
classification_report_for_each_column(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.62      0.36      0.46      1552
          1       0.82      0.93      0.87      4967
          2       0.58      0.31      0.41        35

avg / total       0.77      0.79      0.77      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5466
          1       0.81      0.33      0.47      1088

avg / total       0.87      0.88      0.85      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3869
          1       0.73      0.53      0.61      2685

avg / total       0.73      0.73   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vectorizer', 'tfidf', 'classifer', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'classifer__estimator__bootstrap', 'classifer__estimator__class_weight', 'classifer__estimator__criterion', 'classifer__estimator__max_depth', 'classifer__estimator__max_features', 'classifer__estimator__max_leaf_nodes', 'classifer__estimator__min_impurity_decrease', 'classifer__estimator__min_impurity_split', 'classifer__estimator__min_samples_leaf', 'classifer__estimator__min_samples_split', 'classifer__estimator__min_weight

In [17]:
from sklearn.grid_search import GridSearchCV
parameters = {
    'tfidf__norm':['l2','l1'],
    'classifer__estimator__min_samples_split' : [2,3]
    #'classifer__estimator__n_neighbors' :[3,5,7,9,11],
    #'classifer__estimator__weights' :['uniform','distance'],
    #'classifer__estimator__metric' :['euclidean','manhattan']
}

cv = GridSearchCV(pipeline, parameters)

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'classifer__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [20]:
cv.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'classifer__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [21]:
cv.best_params_

{'classifer__estimator__min_samples_split': 2, 'tfidf__norm': 'l1'}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
from sklearn.externals import joblib
joblib.dump(cv.best_estimator_,'rf_model.pkl')

['rf_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.